In [1]:
import json
from parsivar import FindStems

from preprocessor.normalizer import Normalizer
from preprocessor.tokenizer import Tokenizer

# Load Data

In [2]:
def read_json(path):
    file = open(path)
    data = json.load(file)
    return data

In [3]:
PATH = "D:\Data\Docs\\University\\term7\Inforamtion Retrieval\project\search_engine\Dataset\IR_data_news_5k.json"
input_data = read_json(PATH)

In [4]:
print(input_data.keys())
print(list(input_data.values())[0])


dict_keys(['4092', '64', '3568', '4968', '5731', '8255', '8979', '5437', '10307', '2513', '11739', '4172', '3110', '2440', '6395', '2001', '7742', '8145', '3781', '11600', '3487', '11706', '2410', '1836', '1673', '2957', '11985', '12114', '6664', '4424', '6339', '134', '5952', '296', '2274', '5298', '10441', '6169', '8179', '485', '10506', '2568', '3334', '2938', '5937', '2194', '10273', '6770', '4697', '11248', '5008', '4452', '4415', '9117', '5680', '9193', '6472', '11500', '7235', '155', '2321', '12165', '7790', '3036', '11019', '11531', '4212', '8598', '10776', '9119', '4873', '10107', '7174', '6828', '787', '12149', '8867', '6723', '3142', '9243', '11687', '6824', '12157', '1502', '5848', '5770', '1926', '1883', '4211', '2762', '2182', '6061', '5129', '6983', '10203', '3318', '5475', '3483', '8236', '2460', '7220', '4894', '10077', '10771', '6745', '7911', '9488', '6953', '4558', '8440', '2955', '3526', '348', '1754', '7731', '3102', '2681', '6565', '750', '8464', '7912', '4376', 

In [5]:
print(list(input_data.values())[0].keys())

dict_keys(['title', 'content', 'tags', 'date', 'url', 'category'])


In [6]:
contents = {doc_id: article['content'] for doc_id, article in input_data.items()}

In [7]:
print(len(contents))

5000


In [8]:
print(contents['456'])


به گزارش خبرنگار ورزشی خبرگزاری فارس، سید رضا صالحی امیری پس از برگزاری مجمع عمومی کمیته ملی المپیک اظهار داشت: امروز چهارمین سال از عمر کمیته بود. در مجمع امروز یک گزارش در 24 محور ارائه دادم، خلاصه این بود که معیار ما  در عملکرد شفافیت، قانونگرایی، اخلاق مداری و تلاش برای کاهش تهدیدات، برگزاری عزتمندانه رویدادها و توسعه سخت افزارها و تلاش برای افزایش منابع بودجه ای بود. 480 میلیارد در 4 سال از دولت گرفتیم و 60 درصد را به فدارسیون ها دادیم. 90 میلیارد هزینه پرسنل و بقیه برای کارهای اداری و زیر ساختی هزینه شد. وی ادامه داد: شاخص گذاری انجام شده و اولین فدراسیونی که بیشترین امتیاز را گرفت، کشتی بود که هزار بود و بقیه به ترتیب دریافتی داشتند. در مجموع هر امتیاز 4 میلیون در نظر گرفته شد. خوشبختانه بالای 90 درصد از اعضای مجمع در مورد کارهای‌ها و برنامه رای مثبت دادند.  رئیس کمیته ملی المپیک گفت:  مورد بعدی انتخابات بود. سه سال طول کشید اساسنامه به تصویب ioc, دولت، و مجلس و شورای نگهبان برسد. این را هم بگویم که حداقل برای 10 سال کمیته داری اساسنامه قوی است و هیچ کس نمی تواند به آن ایراد بگ

# Preprocessing

In [9]:
normalizer = Normalizer()
tokenizer = Tokenizer()
stemmer = FindStems()

def preprocess(contents):
    preprocessed_docs = {}
    for doc_id, content in contents.items():
        # normalizing
        normalized_content = normalizer.normalize(content)
        content_tokens = tokenizer.tokenize(normalized_content)
        
        tokens = []
        for token in content_tokens:
            token = stemmer.convert_to_stem(token)

            tokens.append(token)
        
        # tokens of each doc
        preprocessed_docs[doc_id] = tokens
       
    return preprocessed_docs


In [10]:
documents = preprocess(contents)
# documents = dict(sorted(documents.items(), key=lambda item: item[0]))

In [11]:
documents.keys()

dict_keys(['4092', '64', '3568', '4968', '5731', '8255', '8979', '5437', '10307', '2513', '11739', '4172', '3110', '2440', '6395', '2001', '7742', '8145', '3781', '11600', '3487', '11706', '2410', '1836', '1673', '2957', '11985', '12114', '6664', '4424', '6339', '134', '5952', '296', '2274', '5298', '10441', '6169', '8179', '485', '10506', '2568', '3334', '2938', '5937', '2194', '10273', '6770', '4697', '11248', '5008', '4452', '4415', '9117', '5680', '9193', '6472', '11500', '7235', '155', '2321', '12165', '7790', '3036', '11019', '11531', '4212', '8598', '10776', '9119', '4873', '10107', '7174', '6828', '787', '12149', '8867', '6723', '3142', '9243', '11687', '6824', '12157', '1502', '5848', '5770', '1926', '1883', '4211', '2762', '2182', '6061', '5129', '6983', '10203', '3318', '5475', '3483', '8236', '2460', '7220', '4894', '10077', '10771', '6745', '7911', '9488', '6953', '4558', '8440', '2955', '3526', '348', '1754', '7731', '3102', '2681', '6565', '750', '8464', '7912', '4376', 

In [12]:
print(len(documents))

5000


# Positional Inverted Indexing

In [13]:
import heapq
from invertedIndex.pinverted_index import Term

In [14]:
def positional_inverted_indexing(documents: dict):
    dictionary = {}
    for doc_id, tokens in documents.items():
        for position, token in enumerate(tokens):
            if token not in dictionary:
                term = Term()
            else:
                term = dictionary[token]

            term.update_postings_list(doc_id, position)
            dictionary[token] = term

    return dictionary

In [15]:
def delete_the_k_most_repeated_terms(pos_inv_index, k=50):
    max_heap = []
    for term in pos_inv_index.keys():
        heapq.heappush(max_heap, (-pos_inv_index[term].collect_freq, term))

    k_most_repeated = []
    for _ in range(k):
        item = heapq.heappop(max_heap)
        pos_inv_index.pop(item[1])
        k_most_repeated.append(item)

    k_most_repeated = [(-key, value) for key, value in k_most_repeated]

    return pos_inv_index, k_most_repeated


#### creating positional inverted index 
Here I'm creating a positional inverted index and showing an example

In [16]:
pos_inv_index = positional_inverted_indexing(documents=documents)
pos_inv_index, k_most_repeated = delete_the_k_most_repeated_terms(pos_inv_index)

In [17]:
# Save index
from invertedIndex.pinverted_index import InvertedIndex
pii_obj = InvertedIndex(dictionary=pos_inv_index).save(path='pii.pkl')

In [18]:
print(k_most_repeated)

[(87752, 'و'), (66442, 'در'), (53881, 'به'), (37172, 'از'), (33178, 'این'), (30141, 'که'), (27663, 'با'), (26914, 'را'), (22985, 'اس'), (16793, 'کرد&کن'), (12194, 'داشت&دار'), (12159, 'برای'), (11381, 'شد&شو'), (11297, 'تیم'), (10426, 'ان'), (10019, 'کرد'), (9586, 'بود&باش'), (8867, 'هم'), (8764, 'کشور'), (8000, 'شد'), (7644, 'ما'), (7362, 'یک'), (6956, 'بازی'), (6484, 'می'), (6347, 'بر'), (6347, 'های\u200c'), (6341, 'تا'), (6324, 'باید'), (6169, 'وی'), (6044, 'شده'), (5892, 'خواست&خواه'), (5853, 'خود'), (5776, 'مجلس'), (5770, 'گزارش'), (5763, 'داد&ده'), (5728, 'اسلامی'), (5694, 'فارس'), (5499, 'گفت'), (5412, 'پیام'), (5129, 'ایران'), (5108, 'مردم'), (5107, 'رییس'), (5084, 'خبرگزاری'), (5022, 'انتهای'), (4885, 'سال'), (4858, 'اما'), (4845, 'گرفت&گیر'), (4703, 'دولت'), (4350, 'بازیکن'), (4266, 'داشت&دارد')]


In [19]:
pos_inv_index['ورزش'].position_in_docs
# x = {}
# for term, term_obj in pos_inv_index.items():
#     len_postings_list = len(term_obj.get_docs())
#     x[term] = len_postings_list
#     
# print(dict(sorted(x.items(), key=lambda item: item[1])[:3]))

{'3110': [47],
 '2957': [377],
 '5952': [45, 51, 252, 262, 269, 274, 408],
 '4415': [92],
 '4212': [342, 357],
 '4873': [512, 549, 671],
 '6828': [620],
 '4211': [167, 179],
 '3526': [187],
 '4867': [100],
 '1006': [80],
 '524': [476],
 '3473': [22, 112, 163, 204],
 '1204': [171, 395],
 '2998': [8, 24, 89, 134],
 '1849': [435],
 '6370': [98, 162],
 '2616': [299],
 '714': [276],
 '1230': [68],
 '3767': [41, 305, 321, 402, 452],
 '2355': [13, 79, 80, 91, 103, 107, 134, 161, 184, 256, 324, 603, 642],
 '2892': [82, 122, 182, 234, 401],
 '3256': [11],
 '4607': [48, 79, 367],
 '4321': [117],
 '5402': [172],
 '6436': [3212, 3738, 4970, 5044, 5059, 5081, 6136],
 '3081': [146],
 '5611': [95, 126, 245],
 '3045': [40],
 '5959': [35, 69],
 '1263': [909, 1524],
 '324': [536],
 '8856': [4, 90, 110],
 '10082': [32, 75],
 '3357': [63],
 '4653': [51],
 '2188': [342],
 '229': [258],
 '525': [1063],
 '3905': [421],
 '5862': [148, 235, 344],
 '2880': [56, 61],
 '2644': [34, 259, 298, 542, 612],
 '8713': [

For checking correctness of equality of collection frequency and sum of term frequency in all docs 

In [21]:
print(pos_inv_index['ورزش'].collect_freq)
print(sum(pos_inv_index['ورزش'].term_freq_in_docs.values()))

1160
1160


# Documents in Vector Space

In [22]:
import numpy as np
from invertedIndex.vectorspace import calculate_weights_for_all_terms, calculate_idf_for_all_terms

In [23]:
calculate_weights_for_all_terms(dictionary=pos_inv_index, collection_size=len(contents))
calculate_idf_for_all_terms(pos_inv_index, len(pos_inv_index.keys()))

In [24]:
pos_inv_index['ورزش'].weight_in_docs

{'3110': 1.1001794975729038,
 '2957': 1.1001794975729038,
 '5952': 2.0299390346356345,
 '4415': 1.1001794975729038,
 '4212': 1.431366526956876,
 '4873': 1.6250985198717354,
 '6828': 1.1001794975729038,
 '4211': 1.431366526956876,
 '3526': 1.1001794975729038,
 '4867': 1.1001794975729038,
 '1006': 1.1001794975729038,
 '524': 1.1001794975729038,
 '3473': 1.7625535563408483,
 '1204': 1.431366526956876,
 '2998': 1.7625535563408483,
 '1849': 1.1001794975729038,
 '6370': 1.431366526956876,
 '2616': 1.1001794975729038,
 '714': 1.1001794975729038,
 '1230': 1.1001794975729038,
 '3767': 1.8691719657618353,
 '2355': 2.325717135238516,
 '2892': 1.8691719657618353,
 '3256': 1.1001794975729038,
 '4607': 1.6250985198717354,
 '4321': 1.1001794975729038,
 '5402': 1.1001794975729038,
 '6436': 2.0299390346356345,
 '3081': 1.1001794975729038,
 '5611': 1.6250985198717354,
 '3045': 1.1001794975729038,
 '5959': 1.431366526956876,
 '1263': 1.431366526956876,
 '324': 1.1001794975729038,
 '8856': 1.6250985198717

In [25]:
pos_inv_index['ورزش'].term_freq_in_docs

{'3110': 1,
 '2957': 1,
 '5952': 7,
 '4415': 1,
 '4212': 2,
 '4873': 3,
 '6828': 1,
 '4211': 2,
 '3526': 1,
 '4867': 1,
 '1006': 1,
 '524': 1,
 '3473': 4,
 '1204': 2,
 '2998': 4,
 '1849': 1,
 '6370': 2,
 '2616': 1,
 '714': 1,
 '1230': 1,
 '3767': 5,
 '2355': 13,
 '2892': 5,
 '3256': 1,
 '4607': 3,
 '4321': 1,
 '5402': 1,
 '6436': 7,
 '3081': 1,
 '5611': 3,
 '3045': 1,
 '5959': 2,
 '1263': 2,
 '324': 1,
 '8856': 3,
 '10082': 2,
 '3357': 1,
 '4653': 1,
 '2188': 1,
 '229': 1,
 '525': 1,
 '3905': 1,
 '5862': 3,
 '2880': 2,
 '2644': 5,
 '8713': 1,
 '4315': 3,
 '5616': 1,
 '9092': 1,
 '5280': 4,
 '3996': 1,
 '1948': 3,
 '5570': 2,
 '8297': 1,
 '857': 1,
 '6733': 1,
 '4347': 1,
 '6508': 2,
 '454': 3,
 '9616': 1,
 '5767': 5,
 '2984': 5,
 '2562': 1,
 '2391': 1,
 '885': 2,
 '4231': 2,
 '6115': 7,
 '5255': 1,
 '11842': 1,
 '1506': 1,
 '6616': 5,
 '3288': 3,
 '3620': 9,
 '1392': 3,
 '5913': 2,
 '5729': 9,
 '6171': 4,
 '10665': 3,
 '11845': 3,
 '8289': 3,
 '6190': 1,
 '3900': 1,
 '900': 1,
 '786': 

In [26]:
pos_inv_index['فوتبال'].weight_in_docs

{'64': 0.5764264802672645,
 '3568': 0.5764264802672645,
 '4968': 0.5764264802672645,
 '5437': 0.5764264802672645,
 '2513': 0.9234698019781817,
 '2440': 0.5764264802672645,
 '2001': 1.0969914628336401,
 '3781': 0.7499481411227231,
 '3487': 0.7499481411227231,
 '1836': 0.5764264802672645,
 '1673': 0.5764264802672645,
 '4424': 0.5764264802672645,
 '6339': 0.5764264802672645,
 '134': 1.0969914628336401,
 '5952': 0.9234698019781817,
 '296': 0.8514518057860204,
 '5298': 0.9234698019781817,
 '6169': 0.5764264802672645,
 '485': 0.5764264802672645,
 '2568': 0.7499481411227231,
 '2938': 0.5764264802672645,
 '4697': 0.5764264802672645,
 '5008': 0.9793312996790704,
 '4452': 0.5764264802672645,
 '4415': 0.8514518057860204,
 '6472': 0.7499481411227231,
 '155': 0.5764264802672645,
 '4212': 0.5764264802672645,
 '4873': 1.1264771313047763,
 '6828': 0.5764264802672645,
 '3142': 0.5764264802672645,
 '1502': 0.8514518057860204,
 '5770': 0.8514518057860204,
 '1926': 0.5764264802672645,
 '2762': 0.576426480

#### Creating champions list for all terms in dictionary

In [27]:
def create_champ_list(dictionary, k):
    for term in dictionary:
        dictionary[term].create_champ_list(k)

In [28]:
create_champ_list(pos_inv_index, k=50)

In [29]:
pos_inv_index['فوتسال'].get_champ_list()

{'4671': 3.9758040616772496,
 '1306': 3.6276046868528597,
 '4612': 3.6276046868528597,
 '2453': 3.6276046868528597,
 '5304': 3.5616739456765463,
 '3905': 3.409620637914702,
 '6405': 3.409620637914702,
 '4846': 3.2191932811928745,
 '2797': 3.2191932811928745,
 '1730': 3.2191932811928745,
 '84': 3.2191932811928745,
 '1535': 3.2191932811928745,
 '5073': 3.1023893766292785,
 '4562': 3.1023893766292785,
 '5106': 2.964239679569807,
 '4703': 2.964239679569807,
 '2715': 2.964239679569807,
 '3612': 2.795158115343855,
 '6452': 2.795158115343855,
 '4830': 2.795158115343855,
 '2486': 2.795158115343855,
 '0': 2.795158115343855,
 '5005': 2.795158115343855,
 '3266': 2.577174066405698,
 '2391': 2.577174066405698,
 '4213': 2.577174066405698,
 '3970': 2.577174066405698,
 '1780': 2.577174066405698,
 '6421': 2.577174066405698,
 '6384': 2.577174066405698,
 '3457': 2.577174066405698,
 '4850': 2.577174066405698,
 '6444': 2.269942805120275,
 '5350': 2.269942805120275,
 '6323': 2.269942805120275,
 '312': 2.269

#### calculating cosine similarity

In [30]:
from invertedIndex.vectorspace import calculate_tf_for_query, calculate_idf

In [31]:
def calculate_docs_norm(dictionary, collection_size):
    lengths = np.zeros(collection_size)
    for term_obj in dictionary.values():
        for doc_id, weight in term_obj.weight_in_docs.items():
            lengths[int(doc_id)] += weight ** 2
    return np.sqrt(lengths)

In [32]:
collection_size = len(pos_inv_index.keys())
docs_norms = calculate_docs_norm(pos_inv_index, collection_size)
docs_norms

array([10.53805966,  7.95390385,  0.        , ...,  0.        ,
        0.        ,  0.        ])

In [38]:
def calculate_cosine_similarity(query_tokens, dictionary, collection_size, norms):
    scores = {}  # doc_id -> score

    for term in query_tokens:
        w_t_q = calculate_tf_for_query(term, query_tokens) * calculate_idf(dictionary[term], collection_size)
        for doc_id, w_t_d in dictionary[term].get_champ_list().items():
            if doc_id not in scores:
                scores[doc_id] = 0
            scores[doc_id] += w_t_q * w_t_d 
      
    for doc_id in scores:
        scores[doc_id] /= norms[int(doc_id)]
    return scores

## Query Answering

In [39]:
def find_sentences_with_token(content, query_tokens):
    sentences = content.split('.')
    sentences_with_tokens = []

    for sentence in sentences:
        normalized_sentence = normalizer.normalize(sentence)
        if any(token in normalized_sentence for token in query_tokens):
            sentences_with_tokens.append(sentence.strip())

    return sentences_with_tokens

In [40]:
def showing_result(socres, query_tokens):
    for doc_id in socres:
        title = input_data[str(doc_id)]['title']
        url = input_data[str(doc_id)]['url']
        sentences_with_token = find_sentences_with_token(input_data[str(doc_id)]['content'], query_tokens)

        print(f'*************[{doc_id}]*************', end='\n\n')
        print('Title: ', title, end='\n\n')
        print('url: ', url, end='\n\n')
        for sentence in sentences_with_token:
            print(f'[{sentence}]')
        print()


In [41]:
def search(query):
    query_tokens = preprocess({0: query})[0]
    k = 5
    scores = calculate_cosine_similarity(query_tokens, pos_inv_index, collection_size, docs_norms)
    sorted_scores = dict(sorted(scores.items(), key=lambda item: item[1], reverse=True)[:k])
    showing_result(sorted_scores, query_tokens)

#### Simple common single query

In [42]:
query = 'جام جهانی'
search(query)

*************[127]*************

Title:  رئیس فیفا تماشاگر بازی رقبای ایران می شود

url:  https://www.farsnews.ir/news/14001222000684/رئیس-فیفا-تماشاگر-بازی-رقبای-ایران-می-شود

[به گزارش خبرگزاری فارس، فدراسیون فوتبال عراق به صورت رسمی از جانی اینفانتینو رئیس فدراسیون جهانی فوتبال دعوت کرد تا دیدار تیم ملی این کشور با امارات در انتخابی جام جهانی 2022 را از نزدیک تماشا کند]
[عراق یکی از رقبای ایران در انتخابی جام جهانی 2022 به شمار می آید]
[عراقی ها امیدوار هستند در دو بازی باقی مانده از انتخابی جام جهانی برنده شوند و به عنوان سهمیه پلی آف امیدوار به صعود به جام جهانی باشند]
[تیم ملی کشورمان و کره جنوبی در گروه اول انتخابی صعودشان به جام جهانی 2022 قطر قطعی شده است]

*************[1357]*************

Title:  میزبان پلی آف آسیا برای راهیابی به جام جهانی مشخص شد

url:  https://www.farsnews.ir/news/14001206000350/میزبان-پلی-آف-آسیا-برای-راهیابی-به-جام-جهانی-مشخص-شد

[به گزارش خبرگزاری فارس، سایت کنفدراسیون فوتبال آسیا اعلام کرد قطv میزبان بازی پلی آف آسیا برای راهیابی به جام جهانی 2022 خوا

#### Simple common phrase query

In [43]:
query = 'سرمربی فوتسال'
search(query)

*************[5073]*************

Title:  تیم ملی و ناظم الشریعه نامزد برترین های فوتسال جهان

url:  https://www.farsnews.ir/news/14001019000727/تیم-ملی-و-ناظم-الشریعه-نامزد-برترین-های-فوتسال-جهان

[به ‌گزارش خبرنگار ورزش فارس، سایت رسمی فوتسال پلنت که هر ساله برترین های فوتسال جهان را معرفی می کند در سال ۲۰۲۱ هم برترین های فوتسال جهان را معرفی کرد]
[نام سید محمد ناظم الشریعه سرمربی تیم‌ملی فوتسال کشورمان و همچنین تیم ملی فوتسال ایران جزو ۱۰ گزینه نهایی انتخاب برترینهای فوتسال جهان قرار گرفته است]

*************[4502]*************

Title:  برگزاری مراسم معارفه سرمربی تیم ملی زیر 19 سال فوتسال+عکس

url:  https://www.farsnews.ir/news/14001026000632/برگزاری-مراسم-معارفه-سرمربی-تیم-ملی-زیر-9-سال-فوتسال-عکس

[به گزارش خبرگزاری فارس و به نقل از سایت فدراسیون فوتبال، مراسم معارفه علی صانعی سرمربی تیم ملی زیر ۱۹ سال فوتسال امروز با حضور احسان اصولی،رییس کمیته فوتسال و حسین شمس برگزار شد]

*************[4850]*************

Title:  شمسایی سرمربی تیم ملی فوتسال شد، صانعی مربی 19 ساله‌ها

url:  ht

#### Difficult rare single query

In [44]:
query = 'شاملو'
search(query)

*************[11434]*************

Title:  انتخاب اعضای شورای مرکزی حزب «نسل نو»/ تعیین دبیرکل دراولین جلسه

url:  https://www.farsnews.ir/news/14000815000431/انتخاب-اعضای-شورای-مرکزی-حزب-نسل-نو-تعیین-دبیرکل-دراولین-جلسه

[این تشکل جدیدالتاسیس سیاسی با شعار «نسل نو؛ امیدوار، مصلح و تمدن ساز» پا به عرصه سیاست گذاشته است و اسامی شورای مرکزی جدید حزب «نسل نو» به شرح زیر است: ۱- سید احسان قاضی زاده هاشمی  ۲- محسن فتحی ۳- مهدی اقراریان ۴- جعفر بندی شربیانی ۵- سوده نجفی ۶- حامد علامتی ۷- مهدی رمضانی ۸- عطاالله اشرفی اصفهانی ۹- محمدرضاکاشفی ۱۰- محسن میرحاجی ۱۱- امیرابراهیم رسولی ۱۲- حامدزمانی ۱۳- محمدنبی آقاتقی ۱۴- حجت الاسلام میثم آسوپا ۱۵- علی حقیقت ۱۶- نوید خاصه باف ۱۷- امیرخوشخو ۱۸ سید امید میرغضنفری ۱۹- محسن اخلاقی ۲۰- امیدعلی پورحسن ۲۱- سید حامد موسوی ۲۲- حسین محمدیان ۲۳- عادله رضا قلی زاده ۲۴- مازیار عظیم زاده ایرانی ۲۵- سید حسن سیدابراهیم ۲۶- رضا داودی ۲۷- امید شنعه ۲۸- سجاد اکبری ۲۹- آلما فرامرزی ۳۰- علیرضا رحمانی انتخابات بازرسین شورای مرکزی  نیز برگزار و «رضا ترابی»، «حامد توحیدی م

#### Difficult rare phrase query

In [45]:
query = 'قرنطینه کرونا'
search(query)

*************[7375]*************

Title:  تست کرونای رئیس مجلس مثبت اعلام شد

url:  https://www.farsnews.ir/news/14001208001040/تست-کرونای-رئیس-مجلس-مثبت-اعلام-شد

[به گزارش خبرنگار خبرگزاری فارس، تست کرونای محمد باقر قالیباف رئیس مجلس شورای اسلامی مثبت اعلام شد]
[بنا بر اعلام نزدیکان رئیس مجلس، حال وی مساعد بوده و در دفتر کار مجلس، در قرنطینه و در حال پیگیری امور است]

*************[5997]*************

Title:  مدافع بارسلونا به کرونا مبتلا شد

url:  https://www.farsnews.ir/news/14001006000859/مدافع-بارسلونا-به-کرونا-مبتلا-شد

[به گزارش خبرگزاری فارس، تست کرونای کلمنت لنگله مدافع بارسلونا مثبت اعلام شده است]
[بر این اساس این بازیکن باید در قرنطینه قرار بگیرد و از سایر بازیکنان دور باشد]

*************[3660]*************

Title:  تست کرونای نوشاد پس از ۱۰ روز قرنطینه منفی شد

url:  https://www.farsnews.ir/news/14001105000890/تست-کرونای-نوشاد-پس-از-۱۰-روز-قرنطینه-منفی-شد

[به گزارش خبرنگار ورزشی خبرگزاری فارس، نوشاد عالمیان ستاره تنیس روی میز کشورمان که در لیگ فرانسه بازی می‌کند، در چند 